In [23]:
from google.colab import drive
import pandas as pd
import numpy as np
import sys
import seaborn as sns
import os
import matplotlib.pyplot as plt
import random
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.mixture import GaussianMixture
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.neighbors import KernelDensity
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [9]:
random.seed(0)

# https://mjeensung.github.io/characterbigramtfidf/

# Load data
drive.mount('/content/gdrive', force_remount=True)
data_dir = '../content/gdrive/MyDrive/544/data'

data_path = os.path.join(data_dir, "preprocessed_data.csv")

# Load dataset
with open(os.path.join(data_dir, "X_train.npy"), 'rb') as f:
    X_train = np.load(f)
with open(os.path.join(data_dir, "X_val.npy"), 'rb') as f:
    X_val = np.load(f)
with open(os.path.join(data_dir, "X_test.npy"), 'rb') as f:
    X_test = np.load(f)

y_train = pd.read_csv(os.path.join(data_dir, "y_train.csv"))
y_val = pd.read_csv(os.path.join(data_dir, "y_val.csv"))
y_test = pd.read_csv(os.path.join(data_dir, "y_test.csv"))

Mounted at /content/gdrive


In [10]:
# Train/val/test = 75/35/40

In [11]:
X_train.shape

(3750, 5000)

In [12]:
y_train = np.ravel(y_train)
y_train.shape

(3750,)

In [20]:
np.sort(np.unique(y_train))

array(['Broooooke_', 'DarkPiano', 'Djalfy', 'Dogbook', 'DonniesGirl69',
       'Dutchrudder', 'JBnVFCLover786', 'Jayme1988', 'JessMcFlyxxx',
       'Karen230683', 'MTVnHollyWEST23', 'MiDesfileNegro', 'Quimo',
       'SallytheShizzle', 'SongoftheOss', 'Spidersamm', 'StDAY',
       'TessMorris', 'TizBanana', 'TraceyHewins', 'VioletsCRUK',
       'bigenya', 'cookiemonster82', 'enamoredsoul', 'felicityfuller',
       'hollyalyxfinch', 'insearchofnkotb', 'jaybranch', 'karinb_za',
       'kasey79', 'keza34', 'lesley007', 'linnetwoods', 'maynaseric',
       'mcraddictal', 'michxxblc', 'mrs_mcsupergirl', 'nuttychris',
       'patriciaco', 'paul_steele', 'ramdomthoughts', 'rgoodchild',
       'shanajaca', 'shellrawlins', 'thisgoeshere', 'torilovesbradie',
       'tsarnick', 'twebbstack', 'vacant_heart', 'whitsundays'],
      dtype=object)

In [27]:
lda = LinearDiscriminantAnalysis(n_components=49)
X_transformed = lda.fit_transform(X_train, y_train)
X_transformed.shape

ValueError: ignored

In [ ]:
class KDEClassifier(BaseEstimator, ClassifierMixin):
    """Bayesian generative classification based on KDE
    
    Parameters
    ----------
    bandwidth : float
        the kernel bandwidth within each class
    kernel : str
        the kernel name, passed to KernelDensity
    """
    def __init__(self, bandwidth=1.0, kernel='gaussian'):
        self.bandwidth = bandwidth
        self.kernel = kernel
        
    def fit(self, X, y):
        self.classes_ = np.sort(np.unique(y))
        training_sets = [X[y == yi] for yi in self.classes_]
        self.models_ = [KernelDensity(bandwidth=self.bandwidth,
                                      kernel=self.kernel).fit(Xi)
                        for Xi in training_sets]
        self.logpriors_ = [np.log(Xi.shape[0] / X.shape[0])
                           for Xi in training_sets]
        return self
        
    def predict_proba(self, X):
        logprobs = np.array([model.score_samples(X)
                             for model in self.models_]).T
        result = np.exp(logprobs + self.logpriors_)
        return result / result.sum(1, keepdims=True)
        
    def predict(self, X):
        return self.classes_[np.argmax(self.predict_proba(X), 1)]

In [21]:
clf = KDEClassifier()
clf = clf.fit(X_train, y_train)

In [22]:
y_pred_train = clf.predict(X_train)
y_pred_val = clf.predict(X_val)
y_pred_test = clf.predict(X_test)

<ipython-input-8-b7ff28ca2245>:29: RuntimeWarning: invalid value encountered in true_divide
  return result / result.sum(1, keepdims=True)
<ipython-input-8-b7ff28ca2245>:29: RuntimeWarning: invalid value encountered in true_divide
  return result / result.sum(1, keepdims=True)
<ipython-input-8-b7ff28ca2245>:29: RuntimeWarning: invalid value encountered in true_divide
  return result / result.sum(1, keepdims=True)


In [15]:
y_train.shape

(3750,)

In [24]:
print(f"Accuracy on training set: {metrics.accuracy_score(y_train, y_pred_train):.3f}")
print(f"Accuracy on val set: {metrics.accuracy_score(y_val, y_pred_val):.3f}")
print(f"Accuracy on test set: {metrics.accuracy_score(y_test, y_pred_test):.3f}")

print(f"F1 score on val set: {metrics.f1_score(y_val, y_pred_val, average = 'macro'):.3f}")
print(f"F1 score on test set: {metrics.f1_score(y_test, y_pred_test, average = 'macro'):.3f}\n")

Accuracy on training set: 0.020
Accuracy on val set: 0.020
Accuracy on test set: 0.020
F1 score on val set: 0.001
F1 score on test set: 0.001



In [ ]:
print("Classification report for the validation set")
print(classification_report(y_val, y_pred_val))

In [ ]:
print("Classification report for the test set")
print(classification_report(y_test, y_pred_test))